# Cambiamenti climatici

## 1. Data collection
I dati usati in questa applicazione si trovano nella cartella [data](applicazione/data/). 
<ul>
    <li>In original si trovano i file originali trovati su internet</li>
    <li>In modified si trovano i file "processati"</li>
</ul>

## Dati sulle temperature e sulle stazioni geografiche
I file contenenti i dati relativi alle temperature ([temperatures.dat](applicazione/data/original/temperatures.dat)) e alle stazioni geografiche ([stations.inv](applicazione/data/original/stations.inv)) sono stati trovati sul sito [National Centers for Environmental Information](https://www.ncei.noaa.gov/). Sono scaricabili al seguente link: https://www.ncei.noaa.gov/pub/data/ghcn/v4/ghcnm.tavg.latest.qcu.tar.gz <br>


## Dati sui Paesi
I file contenenti informazioni sui Paesi sono stati trovati su due repository di github: <br>
- [fips_to_iso_country_codes.csv](applicazione/data/original/fips_to_iso_country_codes.csv) trovato al seguente link: https://github.com/mysociety/gaze/blob/master/data/fips-10-4-to-iso-country-codes.csv <br>
- [countries_info.csv](applicazione/data/original/countries_info.csv) trovato al seguente link https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv <br>

## 2. Data processing

In [29]:
import pandas as pd
import os

DATA_FOLDER = "applicazione" + os.path.sep + "data" + os.path.sep
DATA_ORIGINAL_FOLDER = DATA_FOLDER + "original"+ os.path.sep
DATA_MODIFIED_FOLDER = DATA_FOLDER + "modified"+ os.path.sep

def aggiungi_linea_iniziale(path_file1, path_file2, headers="-"):
    """Funzione che inserisce nel file path_file2 una linea vuota e il contenuto del file path_file1 """
    with open(path_file1, 'r') as file, open(path_file2, 'a') as file2: 
        originalContent = file.read() 
        file.seek(0, 0) # Prima posizione            
        file2.write(headers + "\n")
        file2.write(originalContent) 

def crea_file_csv(file_originale, file_modificato, list_width, headers=""):
    """Funzione che crea un file CSV partendo da un Fixed Width Text File. 
    In input sono richiesti: 
    - Il path del file originale
    - Il path del file CSV che verra' creat
    - Lista delle lunghezze delle colonne di file_originale"""
    if os.path.exists(file_modificato):
        return
    print("Creo file csv partendo da",file_originale,"...")
    file_tmp = file_originale + "tmp"
    aggiungi_linea_iniziale(file_originale, file_tmp)
    df = pd.read_fwf(file_tmp, widths=list_width)   
    df.to_csv(file_modificato, index=False, header=False)
    if headers != "":
        aggiungi_linea_iniziale(file_modificato, file_modificato, headers)

    os.remove(file_tmp)

### Stazioni e temperature
I file [stations.inv](applicazione/data/original/stations.inv) e [temperatures.dat](applicazione/data/original/temperatures.dat) trovati su internet sono Fixed Width Text File. Ho deciso di modificarli e convertirli in file CSV ([stations_created.csv](applicazione/data/modified/stations_created.csv), [tempetatures_created.csv](applicazione/data/modified/tempetatures_created.csv)) con la seguente struttura: <br>

#### Stations file:




In [31]:
# File con le stazioni
FILE_ORIGINALE_STATIONS = DATA_ORIGINAL_FOLDER + "stations.inv"
FILE_CSV_STATIONS = DATA_MODIFIED_FOLDER + "stations_created.csv"

crea_file_csv(FILE_ORIGINALE_STATIONS, FILE_CSV_STATIONS, [11,9,10,7,31])

# File con le temperature
FILE_ORIGINALE_TEMPERATURES = DATA_ORIGINAL_FOLDER + "temperatures.dat"
FILE_CSV_TEMPERATURES =DATA_MODIFIED_FOLDER + "temperatures_created.csv"

list_width= [11,4,4]
for _ in range(12):
    list_width += [5,1,1,1]

list_headers = ["id", "year","elem"]
for i in range(1,13):
    list_headers += ["month:"+str(i)+" value","month:"+str(i)+" dm_flag","month:"+str(i)+" qc_flag","month:"+str(i)+" ds_flag"]
headers = ",".join(list_headers)
crea_file_csv(FILE_ORIGINALE_TEMPERATURES, FILE_CSV_TEMPERATURES, list_width, headers)

print("Files creati correttamente!")



Creo file csv partendo da applicazione\data\original\stations.inv ...
Files creati correttamente!
